# cKDTree test code

Test implementation of cKDTree to compute the difference between a point and its nearest neighbors.

In [25]:
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path
import re

In [23]:
def read_file(file_path):
    with open(file_path, 'r') as f:
        data = []
        for n,line in enumerate(f,1):
            if n==1:
                line = line.strip()
                line = re.findall(r'"[^"]*"', line) # get ['"var_name1"', '"var_name2"', ...]
                column_labels = [var_name.replace('"','') for var_name in line]
            elif n==2:
                continue
            else:
                data_line = [eval(data) for data in line.split()]
                if len(data_line)==4:
                    break
                data.append(data_line)
    return pd.DataFrame(data, columns=column_labels)

In [22]:
file = Path('/Users/jarl/2d-discharge-nn/data/avg_data/200Vpp_100Pa_node.dat')

In [29]:
df = read_file(file)[['X', 'Y', 'potential (V)']]
df.head()

,X,Y,potential (V)
0,0.182960,0.259570,17.4177
1,0.172908,0.266347,27.8613
2,0.173172,0.254322,18.8931
3,0.176840,0.246447,11.5936
4,0.170305,0.246504,13.0860


In [31]:
X = df.X.to_numpy()  # units: m
Y = df.Y.to_numpy()